In [1]:
import numpy as np
import pandas as pd
import torch 
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split


In [2]:
#read the input
carDf= pd.read_csv('oneHotNormalizedWithLable.csv')
carLable= carDf['Lable']

#drop the Accident Date and Claim Date
carDf.drop(['Lable'],inplace=True,axis=1)
carDf.head()

print(type(carLable[0]))

<class 'numpy.int64'>


In [3]:
carDf= carDf.values
carLable= carLable.values
print(type(carDf))

<class 'numpy.ndarray'>


In [4]:
#build the own data loader


In [5]:
#create the class Dataset which returns the data and labels
from torch.utils.data.dataset import Dataset, TensorDataset
class myDataset(Dataset):
    
    def __init__(self,features,lables,transform= None):
        dataTensor= []
        lableTensor= []
        dataSize= features.shape[0]
        
        for data in range(dataSize):
            feature= features[data,:]
            #feature= torch.from_numpy(feature).float()
            feature= torch.Tensor(feature)
            dataTensor.append(feature)
            
            lable= np.asanyarray(lables[data])
            lable= torch.from_numpy(lable).float()
           
            #lable= torch.Tensor(lable)
            #print("lable:",lable)
            #assert(False)
           
            lableTensor.append(lable)
        
        #put everything in features and lables
        self.features= dataTensor
        self.lables= lableTensor
    
    
    def __len__(self):
        return len(self.features)
    
    def __getitem__(self,index):
        feature= self.features[index]
        lable= self.lables[index]
        #print("get_item feature:",feature)
        #print("get_item lable:",lable)
        return feature,lable

In [6]:
#split the dataset into two part train and test
X_train,X_test,y_train,y_test = train_test_split(carDf,carLable,random_state=3,test_size=0.25)
print('xtrain:',type(X_train))

xtrain: <class 'numpy.ndarray'>


In [7]:
#make the dataset for train and test loader
mytrainDataset= myDataset(X_train,y_train)
mytestDataset= myDataset(X_test,y_test)

In [8]:
feature,lable= mytrainDataset.__getitem__(0)
print("feature:",feature)
print("lable:",lable)
print(feature.size())

feature: tensor([0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.9295, 0.0000, 0.0000,
        0.0000, 0.0000, 0.0000, 0.0000, 0.0705, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0000, 1.0000, 0.0000, 1.0000, 0.0000, 1.0000, 0.0000, 0.0000,
        1.0000, 0.0000, 0.0000, 1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0000, 1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0000, 1.0000, 0.0000, 0.0000, 0.0000, 1.0000, 0.0000, 0.0705,
        0.0000, 0.9295, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 1.0000,
        0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 1.0000, 0.0000,
        0.0000, 1.0000, 0.0000, 1.0000, 0.0000, 1.0000, 0.0000, 0.0000, 0.0000,
        1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 1.0000, 0.0000, 1.0000, 0.0000,
        0.0000, 0.0000, 0.0000, 0.0188, 0.2500, 1.0000, 0.0497])
lable: tensor(1.)
torch.Size([97])


In [9]:
trainLoader= torch.utils.data.DataLoader(mytrainDataset,batch_size=32,shuffle=True,num_workers=0)
testLoader= torch.utils.data.DataLoader(mytestDataset,batch_size=32,shuffle=True,num_workers=0)


In [10]:
#network architecture
class ANN(torch.nn.Module):
    def __init__(self):
        super(ANN,self).__init__()
        self.fc1= nn.Linear(97,6)
        self.fc2= nn.Linear(6,7)
        self.fc3= nn.Linear(7,9)
        self.fc4= nn.Linear(9,4)
        self.fc5= nn.Linear(4,5)
        self.fc6= nn.Linear(5,4)
        self.fc7= nn.Linear(4,4)
        self.fc8= nn.Linear(4,2)
    
    def forward(self,x):
        x= self.fc1(x)
        x= F.relu(x)
        x= self.fc2(x)
        x= F.relu(x)
        x= self.fc3(x)
        x= F.relu(x)
        x= self.fc4(x)
        x= F.relu(x)
        x= self.fc5(x)
        x= F.relu(x)
        x= self.fc6(x)
        x= F.relu(x)
        x= self.fc7(x)
        x= F.relu(x)
        x= self.fc8(x)
        x= F.sigmoid(x)
        return x
        

In [11]:
#code on GPU
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)
model= ANN()
model= model.to(device)

cuda:0


In [12]:
#define a loss function
criterion= nn.CrossEntropyLoss()
optimizer= optim.SGD(model.parameters(),lr=0.01,momentum = 0.9)
criterion= criterion.to(device) 

In [13]:
def testModle(model,epochNo):
    correct= 0
    total= 0
    lableList= []
    predictedList= []
    #lableList= lableList.to(device)
    #predictedList= predictedList.to(device)
    for i,(feature,lable) in enumerate(testLoader,0):
            #gets the inputs
            inputs= feature
            lables= lable
            lables= lables.type(torch.LongTensor)
            
            inputs = inputs.to(device)
            lables = lables.to(device)

            output= model(inputs)
            _,prediction= torch.max(output.data,1)
            correct += (prediction == lables).sum()

            total+= lables.size(0)
            lableList.append(lables)
            predictedList.append(prediction)

       
    
    trueLable= []
    predictedLable= []
    listLength= len(lableList)
    
    for i in range(listLength):
        size= len(lableList[i])
        for j in range(size):
            trueLable.append(lableList[i][j].cpu().numpy())
            predictedLable.append(predictedList[i][j].cpu().numpy())
    totalLength= len(trueLable)
    TP=FP=FN=TN= 0
    for i in range(totalLength):
        if(trueLable[i]==1 and predictedLable[i]==1):
            TP += 1
        elif(trueLable[i]==1 and predictedLable[i]==0):
            FN += 1
        elif(trueLable[i]==0 and predictedLable[i]==0):
            TN += 1
        elif(trueLable[i]==0 and predictedLable[i]==1):
            FP += 1
    print("\nResult for epoch: ",epochNo)
    print("Accuracy:",(TP+TN)/(TP+FP+FN+TN)*100)
    print("Sensitivity:",TP/(TP+FN)*100)
    print("Specificity:",TN/(TN+FP)*100)
    print("\n")
    print("\nTP:",TP)
    print("\nFN:",FN)
    print("\nFP:",FP)
    print("\nTN:",TN)




In [17]:
#tarin the network
for epoch in range(10):
    running_loss = 0.0
    for i,(feature,lable) in enumerate(trainLoader,0):
        #gets the inputs
                
        inputs= torch.tensor(feature)
        lables= torch.tensor(lable)
        lables= lables.type(torch.LongTensor)
        
        inputs = inputs.to(device)
        lables = lables.to(device)
        
        #zero the parameter gradients
        optimizer.zero_grad()
        
        #forward+backward+optimize
        output = model(inputs)
        loss = criterion(output,lables)
        loss.backward()
        optimizer.step()
        
        #print the statistics
        running_loss += loss.item()
        
    #print after every epoch
    print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / X_train.shape[0]))
    testModle(model,epoch+1)
        
print("finished Training")


/usr/local/anaconda35/lib/python3.6/site-packages/torch/nn/functional.py:1006: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")


[1,   680] loss: 0.022

Result for epoch:  1
Accuracy: 49.98620309050772
Sensitivity: 0.0
Specificity: 100.0



TP: 0

FN: 3625

FP: 0

TN: 3623
[2,   680] loss: 0.022

Result for epoch:  2
Accuracy: 50.01379690949227
Sensitivity: 100.0
Specificity: 0.0



TP: 3625

FN: 0

FP: 3623

TN: 0
[3,   680] loss: 0.022

Result for epoch:  3
Accuracy: 50.01379690949227
Sensitivity: 100.0
Specificity: 0.0



TP: 3625

FN: 0

FP: 3623

TN: 0
[4,   680] loss: 0.022

Result for epoch:  4
Accuracy: 49.98620309050772
Sensitivity: 0.0
Specificity: 100.0



TP: 0

FN: 3625

FP: 0

TN: 3623
[5,   680] loss: 0.022

Result for epoch:  5
Accuracy: 49.98620309050772
Sensitivity: 0.0
Specificity: 100.0



TP: 0

FN: 3625

FP: 0

TN: 3623
[6,   680] loss: 0.022

Result for epoch:  6
Accuracy: 49.98620309050772
Sensitivity: 0.0
Specificity: 100.0



TP: 0

FN: 3625

FP: 0

TN: 3623
[7,   680] loss: 0.022

Result for epoch:  7
Accuracy: 49.98620309050772
Sensitivity: 0.0
Specificity: 100.0



TP: 0

FN: 3625

F

In [ ]:
#train on the training set
testModle(model,1);